# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
#turn off warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,5.85,88,62,2.56,AU,1689871303
1,1,waitangi,-43.9535,-176.5597,6.57,93,88,0.89,NZ,1689871006
2,2,bredasdorp,-34.5322,20.0403,9.48,71,9,4.74,ZA,1689871304
3,3,ar rutbah,33.0381,40.2848,36.00,10,0,4.77,IQ,1689871304
4,4,whitehorse,60.7161,-135.0538,15.61,54,75,1.54,CA,1689871176


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
humdisplay = city_data_df.hvplot.points("Lng",
                                        "Lat",
                                        geo = True,
                                        tiles = "OSM",
                                        color="Humidity")
# Display the map
# YOUR CODE HERE
humdisplay

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df
# A max temperature lower than 27 degrees but higher than 21
ideal_weather = ideal_weather.loc[(ideal_weather["Max Temp"]>21 ) & (ideal_weather["Max Temp"]<27 )] 
# Wind speed less than 4.5 m/s
ideal_weather = ideal_weather.loc[(ideal_weather["Wind Speed"]<4.5 )]
# Zero cloudiness
ideal_weather = ideal_weather.loc[(ideal_weather["Cloudiness"]==0) ]                               
# Drop any rows with null values
# Use this website for help https://www.w3schools.com/python/pandas/ref_df_dropna.asp
ideal_weather.dropna()
# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
105,105,newman,37.3138,-121.0208,26.19,19,0,2.45,US,1689871179
212,212,colonia,50.9333,6.9500,24.98,41,0,3.09,DE,1689871147
313,313,deschutes river woods,43.9915,-121.3584,26.48,31,0,3.09,US,1689871353
320,320,marysville,48.0518,-122.1771,25.38,63,0,2.06,US,1689871354
331,331,saint-pierre,-21.3393,55.4781,22.82,78,0,2.06,RE,1689871356
457,457,bafra,41.5678,35.9069,22.23,77,0,3.82,TR,1689871371
465,465,dryden,49.7833,-92.7503,21.16,68,0,3.60,CA,1689871247
501,501,sheridan,44.8333,-106.9173,23.59,49,0,3.60,US,1689871376
511,511,questembert,47.6667,-2.4667,21.46,72,0,4.12,FR,1689871378
531,531,mammoth lakes,37.6485,-118.9721,22.41,38,0,1.54,US,1689871383


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR Code 
hotel_df = pd.DataFrame(ideal_weather,columns=["City","Country","Lng","Lat","Humidity"])
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE for the blank https://www.geeksforgeeks.org/how-to-add-empty-column-to-dataframe-in-pandas/
hotel_df["Hotel Name"]=""
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lng,Lat,Humidity,Hotel Name
105,newman,US,-121.0208,37.3138,19,
212,colonia,DE,6.9500,50.9333,41,
313,deschutes river woods,US,-121.3584,43.9915,31,
320,marysville,US,-122.1771,48.0518,63,
331,saint-pierre,RE,55.4781,-21.3393,78,
457,bafra,TR,35.9069,41.5678,77,
465,dryden,CA,-92.7503,49.7833,68,
501,sheridan,US,-106.9173,44.8333,49,
511,questembert,FR,-2.4667,47.6667,72,
531,mammoth lakes,US,-118.9721,37.6485,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 100000
params = {
    "format":"json",
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
}
# Print a message to follow up the hotel search
print("Starting hotel search")
#place to hold the info
lat = []
lng =[]
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = row["Lat"]
    longitude = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url =  "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
newman - nearest hotel: Hampton Inn & Suites
colonia - nearest hotel: Wasserturm Hotel Cologne
deschutes river woods - nearest hotel: WorldMark Bend - Seventh Mountain Resort
marysville - nearest hotel: Village Inn and Suites
saint-pierre - nearest hotel: Tropic Hotel
bafra - nearest hotel: Kadhırga Otel
dryden - nearest hotel: Holiday Inn Express
sheridan - nearest hotel: Comfort Inn & Suites
questembert - nearest hotel: Auberge Saint-Hernin
mammoth lakes - nearest hotel: Travelodge by Wyndham Mammoth Lakes
cambria - nearest hotel: Olallieberry Inn
okanagan - nearest hotel: Fairfield Inn & Suites
les avirons - nearest hotel: Floralys


,City,Country,Lng,Lat,Humidity,Hotel Name
105,newman,US,-121.0208,37.3138,19,Hampton Inn & Suites
212,colonia,DE,6.9500,50.9333,41,Wasserturm Hotel Cologne
313,deschutes river woods,US,-121.3584,43.9915,31,WorldMark Bend - Seventh Mountain Resort
320,marysville,US,-122.1771,48.0518,63,Village Inn and Suites
331,saint-pierre,RE,55.4781,-21.3393,78,Tropic Hotel
457,bafra,TR,35.9069,41.5678,77,Kadhırga Otel
465,dryden,CA,-92.7503,49.7833,68,Holiday Inn Express
501,sheridan,US,-106.9173,44.8333,49,Comfort Inn & Suites
511,questembert,FR,-2.4667,47.6667,72,Auberge Saint-Hernin
531,mammoth lakes,US,-118.9721,37.6485,38,Travelodge by Wyndham Mammoth Lakes


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display
# Configure the map plot
# YOUR CODE HERE
Hotdisplay = hotel_df.hvplot.points("Lng","Lat", geo = True,tiles = "OSM", color="Hotel Name")
# Display the map
# YOUR CODE HERE
Hotdisplay

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name)